# 利用两端能量$\frac{q_u}{q_d}$确定粒子入射位置$q_x$

## 打开root文件，继承tree中Branch。

In [1]:
%jsroot on
//创建画布
TCanvas *c1=new TCanvas("c1","c1");
//创建时间差histogram
//root中，log（）：e为底；log10（）：10为底；
TH1D *qratio=new TH1D("qratio","log(qu/qd)",160,-0.8,0.8);  

//声明branch变量
Double_t qu, qd, x;
Double_t ln_R = 0;
//Double_t ctof,tof,x;
//int pid;

//打开root文件
TFile *ipf=new TFile("./tree.root");//打开ROOT文件
//得到tree的指针
TTree *tree=(TTree*)ipf->Get("tree");

//将tree中Branch与变量地址联系
tree->SetBranchAddress("qu",&qu);   
tree->SetBranchAddress("qd",&qd);
//tree->SetBranchAddress("ctof",&ctof);
//tree->SetBranchAddress("tof",&tof);
tree->SetBranchAddress("x",&x);
//tree->SetBranchAddress("pid",&pid);

## 遍历root文件事件，抽取$\frac{q_u}{q_d}$的qratio；

In [2]:
Long64_t nentries=tree->GetEntries();//得到事件总数
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    //root中，log（）：e为底；log10（）：10为底；
    qratio->Fill(log(qu/qd));  // if(ng==1) tx->Fill(tu-td), 只写入满足给定条件的事件      
  }
qratio->Draw();
c1->Draw();

## 观察$log\frac{q_u}{q_d}$与$x$关联

In [3]:
tree->Draw("log(qu/qd):x","","colz");
c1->Draw();

## 微分法抽取边界信息
### 左边界

In [4]:
//%jsroot on
TH1D *dqr=new TH1D("dqr","dq/dx",30,-0.8,0.8);//创建qratio的微分histogram，dqr
for(int i=1;i<qratio->GetNbinsX();i++) {//qratio>GetNbinsX（），得到X轴bin总数，由1开始哦
    Double_t dq=qratio->GetBinContent(i+1)-qratio->GetBinContent(i);//qratio>GetBinContent(i+1)得到某个bin对应的constant;
    dqr->Fill(qratio->GetBinLowEdge(i+1),dq);//填图，dtd可看成tdiff的斜率图
}
dqr->Sumw2(0);//不显示传递误差
dqr->Draw();
dqr->Fit("gaus","","",-0.7,-0.35);//高斯拟合histogram
c1->Draw();

 FCN=19.9112 FROM MIGRAD    STATUS=CONVERGED      61 CALLS          62 TOTAL
                     EDM=2.73953e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.37951e+02   1.39909e+01   2.41650e-02  -5.50752e-06
   2  Mean        -5.22143e-01   2.24604e-03   4.59172e-06   1.44856e-02
   3  Sigma        6.19334e-02   1.75803e-03   1.56033e-05  -9.91321e-03


### 右边界

In [5]:
TF1 *f1 = new TF1("f1","[0]*TMath::Exp(-0.5*((x-[1])/[2])^2)",0.35,0.65);//定义函数的方法 TF1
//设置初始参数
//进行参数拟合时，设置合理的初始参数至关重要！
f1->SetParameter(0,-200);//constants
f1->SetParameter(1,0.5);//mean
f1->SetParameter(2,0.05);//sigma
dqr->Fit("f1","R");
dqr->Draw();
c1->SetLogy(0);
c1->Draw();

 FCN=129.144 FROM MIGRAD    STATUS=CONVERGED      78 CALLS          79 TOTAL
                     EDM=2.2593e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -3.54103e+02   1.57951e+01   6.95255e-02  -5.23170e-06
   2  p1           5.28151e-01   2.77317e-03   1.16765e-05   3.13439e-03
   3  p2           5.06528e-02   1.74143e-03   6.47992e-06   1.65802e-02


## 探测器能量比值法刻度位置
 - $t_q =ln\frac{q_u}{q_d}$
 - $t_{ql} = -0.522143$, $t_{qr} = 0.528151$ //左、右边界
 - $t_{qoff} = (t_{ql} + t_{qr})/2 = -0.003$  //中心位置，offset
 $$x = \frac{2L}{t_{qr} - t_{ql}}*(t_q - t_{qoff})=190.4*(t_x + 0.003)$$

## 得到能量比值法刻度后位置图

In [6]:
TH1D *hqx=new TH1D("hqx","hqx",300,-150,150);
TH2D *hdqx=new TH2D("hdqx","hdqx-hx:hx",100,-50,50,150,-150,150);
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对事件进行遍历
    tree->GetEntry(jentry);
    Double_t qx=190.4*(log(qu/qd)+0.003);
    hqx->Fill(qx);//刻度后的tx
    hdqx->Fill(qx-x,x);//二维图，check刻度效果
  }
hqx->Draw();
c1->Draw();

## 检查刻度效果
### qx-x vs x

In [7]:
hdqx->Draw("colz");//为一条竖线，没有关联
c1->Draw();

### tx-x projectionX

In [8]:
TH1D *hdqx1=hdqx->ProjectionX("projx of hdx");
hdqx1->Draw();
hdqx1->Fit("gaus");//如果中心值 mean x偏离0, 说明确定td-tu的边界有问题
c1->Draw();

 FCN=106.157 FROM MIGRAD    STATUS=CONVERGED      61 CALLS          62 TOTAL
                     EDM=6.60119e-13    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.47028e+03   1.34241e+01   5.54977e-02  -8.42054e-08
   2  Mean         5.62783e-01   3.63659e-02   1.83729e-04   3.28377e-06
   3  Sigma        1.14843e+01   2.55769e-02   3.06334e-06  -1.78851e-03


### 小结
- 在用微分法抽取左右边界时，微分法所得HIST图的bin对拟合影响很大；
- offset设置为0时，拟合偏差越小，因为ln（qu/qd）本身就是过0；
- 由于微分所得HIST图bin较大，峰位受拟合区间影响很大，通过调节拟合范围，可以使得qx-x vs x竖直，亦可使投影中心位于零点，但两者都达到完美很难实现；

In [9]:
ipf->Close();//关闭root文件

In [11]:
!jupyter nbconvert homework1.1-2.ipynb --to html

[NbConvertApp] Converting notebook homework1.1-2.ipynb to html

[NbConvertApp] Writing 402162 bytes to homework1.1-2.html

